In [ ]:
!pip uninstall -y fashion_nets
!pip uninstall -y fashion_datasets

!kaggle datasets download -d masterokay/quad-best
!unzip --q ./quad-best.zip

In [1]:
job_worker_name = "g_q_1e5aug_random_building"

In [2]:
#@title Pip Installs & Imports
#!pip uninstall -y fashion_nets
!pip install -q git+https://github.com/NiklasHoltmeyer/FashionNets.git
try:
  from fashionnets.train_jobs.loader.job_loader import prepare_environment, load_job_settings, history_to_csv_string
  from fashionnets.train_jobs.loader.model_loader import load_siamese_model_from_train_job
except:
  from fashionnets.train_jobs.loader.job_loader import prepare_environment, load_job_settings, history_to_csv_string
  from fashionnets.train_jobs.loader.model_loader import load_siamese_model_from_train_job
from fashionnets.train_jobs.loader.checkpoint_loader import load_latest_checkpoint

from fashionnets.models.layer.Augmentation import compose_augmentations
  

In [3]:
#@title Prepare Environment. Download Datasets
from fashionnets.train_jobs.loader.job_loader import add_back_bone_to_train_job

environment, training_job_cfg = prepare_environment(job_worker_name, debugging=False)
import kaggle #<- Requires Secrets, therefore prepare_environment needs to be run before
kaggle.api.authenticate()

kaggle_downloader = kaggle.api.dataset_download_files #<- sadly needs to be injected

train_job = load_job_settings(environment=environment, training_job_cfg=training_job_cfg, kaggle_downloader=kaggle_downloader)

job_settings = add_back_bone_to_train_job(environment=environment, **training_job_cfg)

[2021-11-30 15:52:29,777] {Environment.py:25} DEBUG - Environment: google
[2021-11-30 15:52:29,851] {Environment.py:61} DEBUG - All Paths already exist!
[2021-11-30 15:52:31,417] {attrs.py:77} DEBUG - Creating converter from 3 to 5
[2021-11-30 15:52:31,890] {job_loader.py:73} DEBUG - <function preprocess_input at 0x7fe2eb783290>
[2021-11-30 15:52:31,952] {Environment.py:61} DEBUG - All Paths already exist!
[2021-11-30 15:52:33,864] {job_loader.py:73} DEBUG - <function preprocess_input at 0x7fe2eb783290>
[2021-11-30 15:52:33,931] {Environment.py:61} DEBUG - All Paths already exist!


In [4]:
from fashiondatasets.deepfashion1.DeepFashion1 import DeepFashion1Dataset
from fashionnets.train_jobs.loader.dataset_loader import prepare_ds 
from fashionnets.models.layer.Augmentation import compose_augmentations
import matplotlib.pyplot as plt

def load_model_clean():
  train_job["back_bone"]["info"]["is_triplet"] = True
  train_job["format"] = 'triplet'
  train_job["is_triplet"] = True

  siamese_model_triplet, init_epoch, _callbacks = load_siamese_model_from_train_job(load_weights=False, **train_job)

  train_job["back_bone"]["info"]["is_triplet"] = False
  train_job["format"] = 'quadruplet'
  train_job["is_triplet"] = False

  siamese_model_quadtruplet, init_epoch, _callbacks = load_siamese_model_from_train_job(load_weights=False, **train_job)

  return siamese_model_triplet, siamese_model_quadtruplet

def load_model(cp_path):
  siamese_model_triplet, siamese_model_quadtruplet = load_model_clean()

  train_job["path"]["checkpoint"] = cp_path

  (t_succ, t_init_epoch) = load_latest_checkpoint(siamese_model_triplet, **train_job)
  (q_succ, q_init_epoch) = load_latest_checkpoint(siamese_model_quadtruplet, **train_job)

  assert q_succ ==  t_succ and t_succ
  assert t_init_epoch == q_init_epoch

  print(t_init_epoch-1, "EPOCH")

  return siamese_model_triplet, siamese_model_quadtruplet

def load_ds():
  ds_loader = DeepFashion1Dataset("./deep_fashion_1_256", 
                                  image_suffix="_256", 
                                  model=None, nrows=None,
                                  augmentation=compose_augmentations()(False),
                                  generator_type="apn",
                                  embedding_path="./emb",
                                  hard_sampling=None,
                                  batch_size=32,
                                  n_chunks=6)
  t_ds = ds_loader.load(splits=["test", "val"],
                                is_triplet=True,
                                force_train_recreate=False,
                        force=False, force_hard_sampling=False)
  q_ds = ds_loader.load(splits=["test", "val"],
                                is_triplet=False,
                                force_train_recreate=False,
                        force=False, force_hard_sampling=False)
  
  return t_ds, q_ds

def validate_full(model_, datasets, is_triplet, steps=None):
  test_ds_info, val_ds_info = datasets["test"], datasets["validation"]
  test_ds, val_ds = test_ds_info["dataset"], val_ds_info["dataset"]

  prepare_settings = {
    "input_shape": job_settings["input_shape"],
    "is_triplet": is_triplet,
    "batch_size": 32,
    "augmentation": compose_augmentations(),
    "generator_type": "apn"
  }
  test_ds, val_ds = prepare_ds(test_ds, is_train=False, **prepare_settings), prepare_ds(val_ds, is_train=False, **prepare_settings)
  test_val_ds = test_ds.concatenate(val_ds)
  test_eval = model_.evaluate(test_ds, steps=steps)
  val_eval = model_.evaluate(val_ds, steps=steps)
  test_val_eval = model_.evaluate(test_val_ds, steps=steps)

  return {
    "test": test_eval, "validation": val_eval, "test+validation": test_val_eval
  }

In [5]:
cp_path = "./"
siamese_model_triplet, siamese_model_quadtruplet = load_model(cp_path)
t_ds, q_ds = load_ds()

[2021-11-30 15:52:59,325] {checkpoint_loader.py:47} INFO - Model loaded Weights, Optimizer, History.
[2021-11-30 15:53:02,077] {checkpoint_loader.py:47} INFO - Model loaded Weights, Optimizer, History.
138 EPOCH
[2021-11-30 15:53:02,752] {deep_fashion_1_pairs_generator.py:66} ERROR - WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING 
[2021-11-30 15:53:02,758] {deep_fashion_1_pairs_generator.py:67} ERROR - Model is None. Will only build Random Pairs!
[2021-11-30 15:53:02

test: Map full Paths:   0%|          | 0/4 [00:00<?, ?it/s]

[2021-11-30 15:53:15,662] {io.py:66} INFO - [Load_Split] Elapsed Time: 12.892991304397583s
[2021-11-30 15:53:15,664] {io.py:73} INFO - ************** DeepFashion-DS [DONE] **************
[2021-11-30 15:53:15,667] {io.py:57} INFO - ***************** DeepFashion-DS *****************


val: Map full Paths:   0%|          | 0/4 [00:00<?, ?it/s]

[2021-11-30 15:53:28,834] {io.py:66} INFO - [Load_Split] Elapsed Time: 13.166749477386475s
[2021-11-30 15:53:28,839] {io.py:73} INFO - ************** DeepFashion-DS [DONE] **************
[2021-11-30 15:53:28,842] {io.py:66} INFO - [DF-DS::Load] Elapsed Time: 26.07731866836548s
[2021-11-30 15:53:28,849] {io.py:73} INFO - ************** DeepFashion-DS [DONE] **************
[2021-11-30 15:53:28,851] {io.py:57} INFO - ***************** DeepFashion-DS *****************
[2021-11-30 15:53:28,855] {io.py:57} INFO - ***************** DeepFashion-DS *****************


test: Map full Paths:   0%|          | 0/4 [00:00<?, ?it/s]

[2021-11-30 15:53:41,726] {io.py:66} INFO - [Load_Split] Elapsed Time: 12.87088418006897s
[2021-11-30 15:53:41,728] {io.py:73} INFO - ************** DeepFashion-DS [DONE] **************
[2021-11-30 15:53:41,729] {io.py:57} INFO - ***************** DeepFashion-DS *****************


val: Map full Paths:   0%|          | 0/4 [00:00<?, ?it/s]

[2021-11-30 15:53:55,123] {io.py:66} INFO - [Load_Split] Elapsed Time: 13.39420485496521s
[2021-11-30 15:53:55,125] {io.py:73} INFO - ************** DeepFashion-DS [DONE] **************
[2021-11-30 15:53:55,129] {io.py:66} INFO - [DF-DS::Load] Elapsed Time: 26.278008222579956s
[2021-11-30 15:53:55,133] {io.py:73} INFO - ************** DeepFashion-DS [DONE] **************


In [6]:
triplet_loss = validate_full(siamese_model_triplet, t_ds, is_triplet=True, steps=None)
print(triplet_loss)
quad_loss = validate_full(siamese_model_quadtruplet, q_ds, is_triplet=False, steps=None)
print(quad_loss)


PREPARE
Concatenate
EV TEST
1445/1445 [==============================] - 928s 637ms/step - loss: 0.1086
EV VAL
1490/1490 [==============================] - 952s 639ms/step - loss: 0.1136
EV TEST_VAL
2935/2935 [==============================] - 1872s 638ms/step - loss: 0.1111


In [38]:
from fashionnets.train_jobs.loader.checkpoint_loader import latest_checkpoint, retrieve_epoch_from_checkpoint

latest_cp = latest_checkpoint(cp_path)
epoch = retrieve_epoch_from_checkpoint(latest_cp)

d = {
    "name": train_job["run_name"], "epoch": epoch,"triplet": triplet_loss, "quadtruplet": quad_loss
}

str(d).replace("'", '"')

'{"name": "542_resnet50_imagenet_triplet", "epoch": 138, "triplet": {"test": 0.1086050346493721, "validation": 0.11360414326190948, "test+validation": 0.1111425906419754}, "quadtruplet": {"test": 0.23093846440315247, "validation": 0.23870116472244263, "test+validation": 0.23487883806228638}}'

In [19]:
datasets = t_ds
test_ds_info, val_ds_info = datasets["test"], datasets["validation"]
test_ds, val_ds = test_ds_info["dataset"], val_ds_info["dataset"]

prepare_settings = {
  "input_shape": job_settings["input_shape"],
  "is_triplet": True,
  "batch_size": 32,
  "augmentation": compose_augmentations(),
  "generator_type": "apn"
}

test_ds, val_ds = prepare_ds(test_ds, is_train=False, **prepare_settings), prepare_ds(val_ds, is_train=False, **prepare_settings)
siamese_model_triplet.validate_embedding(list(test_ds.take(1))[0])